# Crossfit Open / Games

With a budding interesting in data analysis/science work. Then about a 8 year hiatus, recently rediscovered crossift at a local box. Thought it would be interesting to look at the works and athletics. 

Additionaly create a model that can predict if an athlete can make using the provided datasets.

All anaysis and modeling will be done with python and tensorflow.

**Datasets**
https://www.kaggle.com/datasets/jeanmidev/crossfit-games?select=2019_opens_scores.csv


**Additional information**

Crossfit 2019 Open Standards - https://games.crossfit.com/workouts/open/2019

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf

print(tf.__version__)

2.9.2


import the datasets

In [1]:
open_scores = pd.read_csv('archive/2020_opens_scores.csv')
open_athletes = pd.read_csv('archive/2020_opens_athletes.csv')

NameError: name 'pd' is not defined

Reviewing the head of open scores dataset

In [19]:
open_scores.head(10)

,affiliate,breakdown,competitorid,division,is_scaled,judge,ordinal,rank,scaled,score,scoredisplay,scoreidentifier,time
0,CrossFit RDU,9 rounds +\n10 wall-ball shots\n,96511,Men (45-49),0,Erin Miller,1,1,0,13520000,352 reps,27f30f9a8c0a564ae799,NaN
1,CrossFit RDU,Within 16 minutes:\n3 rounds +\n25 toes-to-bar...,96511,Men (45-49),0,Harper Thorsen,2,4,0,13420368,342 reps,0ed3d1264f25a8f1890d,NaN
2,CrossFit RDU,200-ft. OH lunge\n50 box step-ups\n50 strict H...,96511,Men (45-49),0,Harper Thorsen,3,1,0,11800018,9:42,f2a143399a330c95321b,582.0
3,CrossFit RDU,132 reps\n6 rounds,96511,Men (45-49),0,Harper Thorsen,4,36,0,11320009,11:51,89101e401c6c85997363,711.0
4,CrossFit RDU,210 reps,96511,Men (45-49),0,Harper Thorsen,5,1,0,12100573,10:27,f7588c9174f1fe90f5c4,627.0
5,CrossFit Teramo,8 rounds +\n19 wall-ball shots\n10-cal. row\n,278581,Men (45-49),0,Mario Andreoli,1,21,0,13330000,333 reps,3bb7595dada1594dd846,NaN
6,CrossFit Teramo,Within 20 minutes:\n4 rounds +\n25 toes-to-bar...,278581,Men (45-49),0,Mario Andreoli,2,1,0,14230090,423 reps,a6b5a4ed4dd4423b0e01,NaN
7,CrossFit Teramo,200-ft. OH lunge\n50 box step-ups\n50 strict H...,278581,Men (45-49),0,Mario Andreoli,3,15,0,11500122,150 reps,892ede2c713597055b40,NaN
8,CrossFit Teramo,132 reps\n6 rounds,278581,Men (45-49),0,Lorenzo Ciampani,4,25,0,11320022,11:38,1469907b6ead663c1d89,698.0
9,CrossFit Teramo,210 reps,278581,Men (45-49),0,Lorenzo Ciampani,5,3,0,12100524,11:16,bc453db91d2de0429ac5,676.0


Quick review of the dataset

- breakdown: description of number of rounds and reps
- competitorid: Linked to the athlet dataset listed below
- ordinal: Listed as the which open workout the athletic preformed
- scoredisplay: Total number of reps preformed by the athlete
- time: Soem workouts recorded in time, other (nan) are listed as max workout

Additional work will have to be spent in the time column. NAN will need to add the max time for the workout/wod

Reviewing the head of the 2019 athletes dataset

In [20]:
open_athletes.head(10)

,competitorid,competitorname,firstname,lastname,postcompstatus,gender,profilepics3key,countryoforigincode,countryoforiginname,divisionid,affiliateid,affiliatename,age,height,weight,overallrank,overallscore,is_scaled,division
0,2536,Samantha Briggs,Samantha,Briggs,accepted,F,0e63d-P2536_14-184.jpg,GB,United Kingdom,19,4098,CrossFit Black Five,37,1.70,61.23,1,33,0,Women (35-39)
1,485089,Renata Pimentel,Renata,Pimentel,accepted,F,04e97-P485089_5-184.jpg,BR,Brazil,19,15868,CrossFit Gurkha,36,1.74,73.00,2,66,0,Women (35-39)
2,16973,Carleen Mathews,Carleen,Mathews,NaN,F,b663a-P16973_6-184.jpg,US,United States,19,10471,CrossFit Saint Helens,35,1.57,62.14,3,101,0,Women (35-39)
3,751083,Danila Capaccetti,Danila,Capaccetti,NaN,F,pukie.png,IT,Italy,19,9329,CrossFit Black Shark,35,1.70,71.00,4,139,0,Women (35-39)
4,313257,Hope Cicero,Hope,Cicero,NaN,F,f204b-P313257_1-184.jpg,US,United States,19,438,CrossFit Billings,36,1.55,61.23,5,176,0,Women (35-39)
5,121354,Corrin Lemieux,Corrin,Lemieux,NaN,F,11635-P121354_1-184.jpg,US,United States,19,18503,Great Woods CrossFit,36,1.65,66.68,6,193,0,Women (35-39)
6,841247,Andreia Pinheiro,Andreia,Pinheiro,NaN,F,55eaf-P841247_5-184.jpg,BR,Brazil,19,4172,BSB CrossFit,36,1.70,72.00,7,204,0,Women (35-39)
7,8015,Rebecca Voigt Miller,Rebecca,Voigt Miller,NaN,F,ca744-P8015_6-184.jpg,US,United States,19,7746,CrossFit Training Yard,38,1.75,68.95,8,209,0,Women (35-39)
8,290001,Ursula Fasel,Ursula,Fasel,NaN,F,f9da1-P290001_8-184.jpg,CH,Switzerland,19,3057,CrossFit Turicum,38,1.64,62.00,9,213,0,Women (35-39)
9,9440,Anna Tobias,Anna,Tobias,NaN,F,8bf0d-P9440_13-184.jpg,US,United States,19,16998,T2 CrossFit,36,1.67,64.86,10,221,0,Women (35-39)


Review of the athletes dataset
- competitorid: used in the open dataset to link the athletes to the workout
- height: measured in meters
- height: measured in KGs
- postcompstatus: need additional information what this column is